# Load ALpha Pose Skeleton

In [1]:
%matplotlib widget


In [2]:
import json 
  
# Opening JSON file 
f = open('alphapose-results.json',) 
  
# returns JSON object as  
# a dictionary 
data = json.load(f) 
  

# Closing file 
f.close() 

In [3]:
data[0]

{'image_id': '0.jpg',
 'category_id': 1,
 'keypoints': [605.4094848632812,
  548.707763671875,
  0.36969053745269775,
  608.5797729492188,
  520.175048828125,
  0.3233851194381714,
  589.5579223632812,
  548.707763671875,
  0.33143749833106995,
  608.5797729492188,
  494.81268310546875,
  0.22594250738620758,
  538.8331298828125,
  529.6859741210938,
  0.21328745782375336,
  545.1737670898438,
  450.428466796875,
  0.05634784325957298,
  602.2391357421875,
  596.26220703125,
  0.3177466094493866,
  538.8331298828125,
  342.6383056640625,
  0.39356890320777893,
  535.662841796875,
  352.149169921875,
  0.39212915301322937,
  542.00341796875,
  244.3589630126953,
  0.23433555662631989,
  538.8331298828125,
  244.3589630126953,
  0.29892778396606445,
  665.6451416015625,
  421.8957824707031,
  0.3502024710178375,
  687.8372192382812,
  501.15325927734375,
  0.3570476174354553,
  725.880859375,
  310.935302734375,
  0.4797382950782776,
  782.9462280273438,
  488.4720458984375,
  0.52755850

In [4]:
len(data)

2658

In [5]:
len(data[0]['keypoints'])

51

In [6]:
data

[{'image_id': '0.jpg',
  'category_id': 1,
  'keypoints': [605.4094848632812,
   548.707763671875,
   0.36969053745269775,
   608.5797729492188,
   520.175048828125,
   0.3233851194381714,
   589.5579223632812,
   548.707763671875,
   0.33143749833106995,
   608.5797729492188,
   494.81268310546875,
   0.22594250738620758,
   538.8331298828125,
   529.6859741210938,
   0.21328745782375336,
   545.1737670898438,
   450.428466796875,
   0.05634784325957298,
   602.2391357421875,
   596.26220703125,
   0.3177466094493866,
   538.8331298828125,
   342.6383056640625,
   0.39356890320777893,
   535.662841796875,
   352.149169921875,
   0.39212915301322937,
   542.00341796875,
   244.3589630126953,
   0.23433555662631989,
   538.8331298828125,
   244.3589630126953,
   0.29892778396606445,
   665.6451416015625,
   421.8957824707031,
   0.3502024710178375,
   687.8372192382812,
   501.15325927734375,
   0.3570476174354553,
   725.880859375,
   310.935302734375,
   0.4797382950782776,
   782.946

In [7]:
import torch
import re
import collections
from torch._six import string_classes, int_classes
import cv2
import time
import numpy as np
import math
import matplotlib.pyplot as plt
import torch.nn as nn

def euclidean_dist(a, b):
      # This function calculates the euclidean distance between 2 point in 2-D coordinates
      # if one of two points is (0,0), dist = 0
      # a, b: input array with dimension: m, 2
      # m: number of samples
      # 2: x and y coordinate
    try:
        if (a.shape[1] == 2 and a.shape == b.shape):
            # check if element of a and b is (0,0)
            bol_a = (a[:,0] != 0).astype(int)
            bol_b = (b[:,0] != 0).astype(int)
            dist = np.linalg.norm(a-b, axis=1)
            return((dist*bol_a*bol_b).reshape(a.shape[0],1))
    except:
        print("[Error]: Check dimension of input vector")
        return 0
    

def data_converter(data):
    data=data['keypoints']
    keypoints=[]
    kp_score=[]
    for a in range (0,len(data)):
        keypoint=[]
        score=[]
        if ((a+3)%3==0):
            keypoint=[data[a],data[a+1]]
            keypoints.append(keypoint)
        elif((a+1)%3==0):
            score=[data[a]]
            kp_score.append(score)

    result_dict={}
    result_dict['keypoints']=torch.FloatTensor(keypoints)
    result_dict['kp_score']=torch.FloatTensor(kp_score)
    return result_dict
    
def vis_frame_fast(data, format='coco'):
    '''
    frame: frame image
    im_res: im_res of predictions
    format: coco or mpii

    return rendered image
    '''
    
    data=data_converter(data)
    
    height=720
    width=1280
    blank_image = np.zeros((height,width,3), np.uint8)

    if format == 'coco':
        l_pair = [
            (0, 1), (0, 2), (1, 3), (2, 4),  # Head
            (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),
            (17, 11), (17, 12),  # Body
            (11, 13), (12, 14), (13, 15), (14, 16)
        ]
        p_color = [(0, 255, 255), (0, 191, 255),(0, 255, 102),(0, 77, 255), (0, 255, 0), #Nose, LEye, REye, LEar, REar
                    (77,255,255), (77, 255, 204), (77,204,255), (191, 255, 77), (77,191,255), (191, 255, 77), #LShoulder, RShoulder, LElbow, RElbow, LWrist, RWrist
                    (204,77,255), (77,255,204), (191,77,255), (77,255,191), (127,77,255), (77,255,127), (0, 255, 255)] #LHip, RHip, LKnee, Rknee, LAnkle, RAnkle, Neck
        line_color = [(0, 215, 255), (0, 255, 204), (0, 134, 255), (0, 255, 50), 
                    (77,255,222), (77,196,255), (77,135,255), (191,255,77), (77,255,77), 
                    (77,222,255), (255,156,127), 
                    (0,127,255), (255,127,77), (0,77,255), (255,77,36)]
    elif format == 'mpii':
        l_pair = [
            (8, 9), (11, 12), (11, 10), (2, 1), (1, 0),
            (13, 14), (14, 15), (3, 4), (4, 5),
            (8, 7), (7, 6), (6, 2), (6, 3), (8, 12), (8, 13)
        ]
        p_color = [PURPLE, BLUE, BLUE, RED, RED, BLUE, BLUE, RED, RED, PURPLE, PURPLE, PURPLE, RED, RED,BLUE,BLUE]
    elif format =='human36':
        l_pair = [
            (0, 1), (0, 2), (1, 3), (2, 4),  # Head
            (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),
            (17, 11), (17, 12),  # Body
            (11, 13), (12, 14), (13, 15), (14, 16)
        ]
        p_color = [(0, 255, 255), (0, 191, 255),(0, 255, 102),(0, 77, 255), (0, 255, 0), #Nose, LEye, REye, LEar, REar
                    (77,255,255), (77, 255, 204), (77,204,255), (191, 255, 77), (77,191,255), (191, 255, 77), #LShoulder, RShoulder, LElbow, RElbow, LWrist, RWrist
                    (204,77,255), (77,255,204), (191,77,255), (77,255,191), (127,77,255), (77,255,127), (0, 255, 255)] #LHip, RHip, LKnee, Rknee, LAnkle, RAnkle, Neck
        line_color = [(0, 215, 255), (0, 255, 204), (0, 134, 255), (0, 255, 50), 
                    (77,255,222), (77,196,255), (77,135,255), (191,255,77), (77,255,77), 
                    (77,222,255), (255,156,127), 
                    (0,127,255), (255,127,77), (0,77,255), (255,77,36)]
    else:
        NotImplementedError


    part_line = {}
    kp_preds = data['keypoints']
    kp_scores = data['kp_score']
    kp_preds = torch.cat((kp_preds, torch.unsqueeze((kp_preds[5,:]+kp_preds[6,:])/2,0)))
    kp_scores = torch.cat((kp_scores, torch.unsqueeze((kp_scores[5,:]+kp_scores[6,:])/2,0)))
    # Draw keypoints
    for n in range(kp_scores.shape[0]):
        if kp_scores[n] <= 0.05:
            continue
        cor_x, cor_y = int(kp_preds[n, 0]), int(kp_preds[n, 1])
        part_line[n] = (cor_x, cor_y)
        cv2.circle(blank_image, (cor_x, cor_y), 4, p_color[n], 3)
    # Draw limbs
    for i, (start_p, end_p) in enumerate(l_pair):
        if start_p in part_line and end_p in part_line:
            start_xy = part_line[start_p]
            end_xy = part_line[end_p]
            cv2.line(blank_image, start_xy, end_xy, line_color[i], 4)
    return blank_image


In [8]:
img=vis_frame_fast(data[500])

In [9]:
img[400]

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=uint8)

In [10]:
plt.imshow(img)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Load Dataset From Human3.6M

# Load Stat File

In [11]:
stat_file='/media/ivan/Ivan/Project Lab/THESIS_FALL_DETECTION/2D to 3D Pose/3d_pose_baseline_pytorch/data/stat_3d.pth.tar'
stat_ = torch.load(stat_file)


In [12]:
stat_['mean'].shape

(96,)

In [13]:
stat_['std'].shape

(96,)

In [14]:
stat_['dim_use']

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 18, 19, 20, 21, 22, 23, 24, 25,
       26, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 51, 52, 53, 54,
       55, 56, 57, 58, 59, 75, 76, 77, 78, 79, 80, 81, 82, 83])

In [15]:
stat_

{'mean': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -2.55652589e-01,
        -7.11960573e+00, -9.81433058e-01, -5.65463051e+00,  3.19636009e+02,
         7.19329269e+01, -1.01705840e+01,  6.91147892e+02,  1.55352986e+02,
        -1.15560633e+01,  7.42149725e+02,  1.66477287e+02, -1.18447102e+01,
         7.36763064e+02,  1.65182437e+02,  2.55651315e-01,  7.11954604e+00,
         9.81423860e-01, -5.09729780e+00,  3.27040413e+02,  7.22258095e+01,
        -9.99656606e+00,  7.08277383e+02,  1.58016408e+02, -1.12642400e+01,
         7.48636864e+02,  1.66665977e+02, -1.14090840e+01,  7.36435064e+02,
         1.63713810e+02,  1.21660845e-03, -8.60110629e-02, -1.93000576e-02,
         2.90583676e+00, -2.11363307e+02, -4.74210915e+01,  5.67537804e+00,
        -4.35088906e+02, -9.76974016e+01,  5.93884964e+00, -4.91891970e+02,
        -1.10666618e+02,  7.37352083e+00, -5.83948619e+02, -1.31171400e+02,
         5.67537804e+00, -4.35088906e+02, -9.76974016e+01,  5.41920653e+00,
    

## Load Dataset pth file

In [16]:
train_inp, train_out, test_inp, test_out = [], [], [], []
train_meta, test_meta = [], []

In [17]:
train_2d_file = '/media/ivan/Ivan/Project Lab/THESIS_FALL_DETECTION/2D to 3D Pose/3d_pose_baseline_pytorch/data/train_2d.pth.tar'
train_3d_file = '/media/ivan/Ivan/Project Lab/THESIS_FALL_DETECTION/2D to 3D Pose/3d_pose_baseline_pytorch/data/train_3d.pth.tar'


In [18]:
train_2d = torch.load(train_2d_file)
train_3d = torch.load(train_3d_file)


In [19]:
for k2d in train_2d.keys():
    (sub, act, fname) = k2d
    print(act)
    k3d = k2d
    k3d = (sub, act, fname[:-3]) if fname.endswith('-sh') else k3d
    num_f, _ = train_2d[k2d].shape
    assert train_3d[k3d].shape[0] == train_2d[k2d].shape[0], '(training) 3d & 2d shape not matched'
    for i in range(num_f):
        train_inp.append(train_2d[k2d][i])
        train_out.append(train_3d[k3d][i])


Directions
Directions
Directions
Directions
Directions
Directions
Directions
Directions
Discussion
Discussion
Discussion
Discussion
Discussion
Discussion
Discussion
Discussion
Eating
Eating
Eating
Eating
Eating
Eating
Eating
Eating
Greeting
Greeting
Greeting
Greeting
Greeting
Greeting
Greeting
Greeting
Phoning
Phoning
Phoning
Phoning
Phoning
Phoning
Phoning
Phoning
Photo
Photo
Photo
Photo
Photo
Photo
Photo
Photo
Posing
Posing
Posing
Posing
Posing
Posing
Posing
Posing
Purchases
Purchases
Purchases
Purchases
Purchases
Purchases
Purchases
Purchases
Sitting
Sitting
Sitting
Sitting
Sitting
Sitting
Sitting
Sitting
SittingDown
SittingDown
SittingDown
SittingDown
SittingDown
SittingDown
SittingDown
SittingDown
Smoking
Smoking
Smoking
Smoking
Smoking
Smoking
Smoking
Smoking
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
WalkDog
WalkDog
WalkDog
WalkDog
WalkDog
WalkDog
WalkDog
WalkDog
WalkTogether
WalkTogether
WalkTogether
WalkTogether
WalkTogether
WalkTogether
WalkTogether
WalkT

In [20]:
k2d

(8, 'Walking', 'Walking.60457274.h5')

In [21]:
train_2d[k2d].shape

(3695, 32)

In [22]:
train_inp[0].shape

(32,)

In [23]:
inputs1 = torch.from_numpy(train_inp[0]).float()

In [24]:
inputs1.shape

torch.Size([32])

In [25]:
inputs1

tensor([-0.5421,  0.3977, -0.2589,  0.4643, -0.4268,  0.7369, -0.1919,  0.8265,
        -0.8474,  0.3264, -0.7130,  0.8766, -0.5754,  1.0041, -0.4103,  0.3898,
        -0.4773,  0.3004, -0.4337,  0.1910, -0.7577,  0.4046, -0.9666,  0.5337,
        -1.1160,  0.7342, -0.2088,  0.3555, -0.0908,  0.5050, -0.1309,  0.8436])

# Load Dataset Using Dataset Loader

In [26]:
from src.datasets.human36m import Human36M
from torch.utils.data import DataLoader
import src.misc as misc
import src.data_process as data_process
from torch.autograd import Variable


actions = misc.define_actions('All')


In [27]:
actions

['Directions',
 'Discussion',
 'Eating',
 'Greeting',
 'Phoning',
 'Photo',
 'Posing',
 'Purchases',
 'Sitting',
 'SittingDown',
 'Smoking',
 'Waiting',
 'WalkDog',
 'Walking',
 'WalkTogether']

In [28]:
train_loader = DataLoader(
    dataset=Human36M(actions=actions, data_path='data/', use_hg=False),
    batch_size=1024,
    shuffle=False,
    num_workers=8,
    pin_memory=True)

test_loader = DataLoader(
    dataset=Human36M(actions=actions, data_path='data/', use_hg=False,is_train=False),
    batch_size=1024,
    shuffle=False,
    num_workers=8,
    pin_memory=True)

In [29]:
train_loader.dataset.train_inp

[array([-0.54206337,  0.39774939, -0.25891828,  0.46426139, -0.42682627,
         0.73693037, -0.19194076,  0.82654666, -0.84738962,  0.32642661,
        -0.71295627,  0.87656733, -0.57538611,  1.00414923, -0.41027526,
         0.38981167, -0.47734803,  0.30042309, -0.43366143,  0.19098921,
        -0.75767234,  0.40461858, -0.96655618,  0.53374401, -1.11597276,
         0.73422003, -0.20879939,  0.35547653, -0.09075686,  0.50499701,
        -0.13092847,  0.84361908]),
 array([-0.54202631,  0.3979099 , -0.25901394,  0.46427406, -0.42701587,
         0.73692873, -0.19194186,  0.82661217, -0.84722452,  0.32673437,
        -0.71286602,  0.87667424, -0.57539653,  1.00411009, -0.40976835,
         0.38978359, -0.47730073,  0.30042956, -0.43329337,  0.19095277,
        -0.75719897,  0.40530494, -0.96662521,  0.5350939 , -1.11449713,
         0.73389111, -0.20857186,  0.35499453, -0.08959195,  0.50466484,
        -0.13451224,  0.84106828]),
 array([-0.54197441,  0.39812588, -0.25907457,  0.46

In [30]:
test_loader.dataset.test_inp[0].shape #Input 2D

(32,)

In [31]:
test_loader.dataset.test_out[0].shape #Input 2D

(48,)

## Unnormalize Data and Visualize Data

  H36M_NAMES = ['']*32 <br>
  H36M_NAMES[0]  = 'Hip' <br>
  H36M_NAMES[1]  = 'RHip'<br>
  H36M_NAMES[2]  = 'RKnee'<br>
  H36M_NAMES[3]  = 'RFoot'<br>
  H36M_NAMES[4]  = 'RFootTip'<br>
  H36M_NAMES[6]  = 'LHip'<br>
  H36M_NAMES[7]  = 'LKnee'<br>
  H36M_NAMES[8]  = 'LFoot'<br>
  H36M_NAMES[12] = 'Spine'<br>
  H36M_NAMES[13] = 'Thorax'<br>
  H36M_NAMES[14] = 'Neck/Nose'<br>
  H36M_NAMES[15] = 'Head'<br>
  H36M_NAMES[17] = 'LShoulder'<br>
  H36M_NAMES[18] = 'LElbow'<br>
  H36M_NAMES[19] = 'LWrist'<br>
  H36M_NAMES[25] = 'RShoulder'<br>
  H36M_NAMES[26] = 'RElbow'<br>
  H36M_NAMES[27] = 'RWrist'<br>


inps is input 2D
tars is label 3D

Normalized 3D data has shape of [48,] because only 16 join are used.<br>
After already unnormalized, 3D data will have [96,],because it will converted to state when all 32 join used before normalized.<br>
Unnormalized data is necessary for visualization and inference<br>

For calculate error, the dimension that ignored will be removed so in this case it will inference using shape [51,]

## Function to Unnormalize Data

In [32]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from __future__ import division

import numpy as np


def unNormalizeData(normalized_data, data_mean, data_std, dimensions_to_use):
    T = normalized_data.shape[0]  # Batch size
    D = data_mean.shape[0]  # 96

    orig_data = np.zeros((T, D), dtype=np.float32)

    orig_data[:, dimensions_to_use] = normalized_data

    # Multiply times stdev and add the mean
    stdMat = data_std.reshape((1, D))
    stdMat = np.repeat(stdMat, T, axis=0)
    meanMat = data_mean.reshape((1, D))
    meanMat = np.repeat(meanMat, T, axis=0)
    orig_data = np.multiply(orig_data, stdMat) + meanMat
    return orig_data

In [33]:


for i, (inps, tars) in enumerate(test_loader):
    inputs = Variable(inps.cuda())
    targets = Variable(tars.cuda(non_blocking=True))

    tars = targets
    ins=inputs

    # Normalize 3D
    # Use targets_unnorm for visualization
    targets_unnorm = data_process.unNormalizeData(tars.data.cpu().numpy(), stat_['mean'], stat_['std'], stat_['dim_use'])
    
    # remove dim ignored
    dim_use = np.hstack((np.arange(3), stat_['dim_use']))
    
    targets_use = targets_unnorm[:, dim_use]
    


In [34]:
test_loader.dataset.test_inp[0].shape #This is normalized dataset

(32,)

In [35]:
targets_unnorm[0].shape # This is unnormalized dataset for visualization

(96,)

In [36]:
targets_use[0].shape # This is unnormalized dataset with dim ignored removed for inference

(51,)

## Function for Visualizing

In [42]:
import src.data_utils as data_utils


def show3Dpose(channels, ax, lcolor="#3498db", rcolor="#e74c3c", add_labels=False): # blue, orange
    """
    Visualize a 3d skeleton

    Args
    channels: 96x1 vector. The pose to plot.
    ax: matplotlib 3d axis to draw on
    lcolor: color for left part of the body
    rcolor: color for right part of the body
    add_labels: whether to add coordinate labels
    Returns
    Nothing. Draws on ax.
    """


    assert channels.size == len(data_utils.H36M_NAMES)*3, "channels should have 96 entries, it has %d instead" % channels.size
    vals = np.reshape( channels, (len(data_utils.H36M_NAMES), -1) )

    I   = np.array([1,2,3,1,7,8,1, 13,14,15,14,18,19,14,26,27])-1 # start points
    J   = np.array([2,3,4,7,8,9,13,14,15,16,18,19,20,26,27,28])-1 # end points
    LR  = np.array([1,1,1,0,0,0,0, 0, 0, 0, 0, 0, 0, 1, 1, 1], dtype=bool)

    # Make connection matrix
    for i in np.arange( len(I) ):
        x, y, z = [np.array( [vals[I[i], j], vals[J[i], j]] ) for j in range(3)]
        ax.plot(x, y, z, lw=2, c=lcolor if LR[i] else rcolor)

    RADIUS = 750 # space around the subject
    xroot, yroot, zroot = vals[0,0], vals[0,1], vals[0,2]
    ax.set_xlim3d([-RADIUS+xroot, RADIUS+xroot])
    ax.set_zlim3d([-RADIUS+zroot, RADIUS+zroot])
    ax.set_ylim3d([-RADIUS+yroot, RADIUS+yroot])

    if add_labels:
        ax.set_xlabel("x")
        ax.set_ylabel("y")
        ax.set_zlabel("z")

    # Get rid of the ticks and tick labels
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])

    ax.get_xaxis().set_ticklabels([])
    ax.get_yaxis().set_ticklabels([])
    ax.set_zticklabels([])

    # Get rid of the panes (actually, make them white)
    white = (1.0, 1.0, 1.0, 0.0)
    ax.w_xaxis.set_pane_color(white)
    ax.w_yaxis.set_pane_color(white)
    # Keep z pane

    # Get rid of the lines in 3d
    ax.w_xaxis.line.set_color(white)
    ax.w_yaxis.line.set_color(white)
    ax.w_zaxis.line.set_color(white)
    
def show2Dpose(channels, ax, lcolor="#3498db", rcolor="#e74c3c", add_labels=False):
    """Visualize a 2d skeleton

    Args
    channels: 64x1 vector. The pose to plot.
    ax: matplotlib axis to draw on
    lcolor: color for left part of the body
    rcolor: color for right part of the body
    add_labels: whether to add coordinate labels
    Returns
    Nothing. Draws on ax.
    """

    assert channels.size == len(data_utils.H36M_NAMES)*2, "channels should have 64 entries, it has %d instead" % channels.size
    vals = np.reshape( channels, (len(data_utils.H36M_NAMES), -1) )

    I  = np.array([1,2,3,1,7,8,1, 13,14,14,18,19,14,26,27])-1 # start points
    J  = np.array([2,3,4,7,8,9,13,14,16,18,19,20,26,27,28])-1 # end points
    LR = np.array([1,1,1,0,0,0,0, 0, 0, 0, 0, 0, 1, 1, 1], dtype=bool)

    # Make connection matrix
    for i in np.arange( len(I) ):
        x, y = [np.array( [vals[I[i], j], vals[J[i], j]] ) for j in range(2)]
        ax.plot(x, y, lw=2, c=lcolor if LR[i] else rcolor)

    # Get rid of the ticks
    ax.set_xticks([])
    ax.set_yticks([])

    # Get rid of tick labels
    ax.get_xaxis().set_ticklabels([])
    ax.get_yaxis().set_ticklabels([])

    RADIUS = 350 # space around the subject
    xroot, yroot = vals[0,0], vals[0,1]
    ax.set_xlim([-RADIUS+xroot, RADIUS+xroot])
    ax.set_ylim([-RADIUS+yroot, RADIUS+yroot])
    if add_labels:
        ax.set_xlabel("x")
        ax.set_ylabel("z")

    ax.set_aspect('equal')

## Visualize 3D Dataset

In [43]:
# Visualize random samples
import matplotlib.gridspec as gridspec




# 1080p	= 1,920 x 1,080
fig = plt.figure( figsize=(19.2, 10.8) )

gs1 = gridspec.GridSpec(5, 9) # 5 rows, 9 columns
gs1.update(wspace=-0.00, hspace=0.05) # set the spacing between axes.
plt.axis('off')

subplot_idx, exidx = 1, 1

# Plot 3d gt
targets_unnorm = data_process.unNormalizeData(test_loader.dataset.test_out[1000], stat_['mean'], stat_['std'], stat_['dim_use'])
ax2 = plt.subplot(gs1[subplot_idx], projection='3d')
p3d = targets_unnorm[0,:]
show3Dpose( p3d, ax2 )


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Visualize 2D Dataset

In [ ]:
# Visualize random samples
import matplotlib.gridspec as gridspec

# 1080p	= 1,920 x 1,080
fig = plt.figure( figsize=(19.2, 10.8) )

gs1 = gridspec.GridSpec(5, 9) # 5 rows, 9 columns
gs1.update(wspace=-0.00, hspace=0.05) # set the spacing between axes.
plt.axis('off')

subplot_idx, exidx = 1, 1

# Plot 2d gt
# wait until we could get the 2D_stat
targets_unnorm = data_process.unNormalizeData(test_loader.dataset.test_out[2048], stat_['mean'], stat_['std'], stat_['dim_use'])
ax2 = plt.subplot(gs1[subplot_idx], projection='3d')
p3d = targets_unnorm[0,:]
show3Dpose( p3d, ax2 )


## Inference Preparation

In [46]:
from src.model import LinearModel, weight_init
import torch.nn as nn
import src.utils as utils
from src import Bar


model_path='/media/ivan/Ivan/Project Lab/THESIS_FALL_DETECTION/2D to 3D Pose/3d_pose_baseline_pytorch/checkpoint/test/ckpt_best.pth.tar'
# create model
print(">>> creating model")
model = LinearModel()
model = model.cuda()
model.apply(weight_init)
print(">>> total params: {:.2f}M".format(sum(p.numel() for p in model.parameters()) / 1000000.0))
criterion = nn.MSELoss(size_average=True).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-3)

print(">>> loading ckpt from '{}'".format('/media/ivan/Ivan/Project Lab/THESIS_FALL_DETECTION/2D to 3D Pose/3d_pose_baseline_pytorch/checkpoint/test/ckpt_best.pth.tar'))
ckpt = torch.load(model_path)
start_epoch = ckpt['epoch']
err_best = ckpt['err']
glob_step = ckpt['step']
lr_now = ckpt['lr']
model.load_state_dict(ckpt['state_dict'])
optimizer.load_state_dict(ckpt['optimizer'])
print(">>> ckpt loaded (epoch: {} | err: {})".format(start_epoch, err_best))
    
def test(test_loader, model, criterion, stat_3d, procrustes=False):
    losses = utils.AverageMeter()
    model.eval()

    all_dist = []
    pred_result=[]
    start = time.time()
    batch_time = 0
    bar = Bar('>>>', fill='>', max=len(test_loader))

    for i, (inps, tars) in enumerate(test_loader):
        inputs = Variable(inps.cuda())
        targets = Variable(tars.cuda(non_blocking=True))

        outputs = model(inputs)

        # calculate loss
        outputs_coord = outputs
        loss = criterion(outputs_coord, targets)

        losses.update(loss.item(), inputs.size(0))

        tars = targets

        # calculate erruracy
        targets_unnorm = data_process.unNormalizeData(tars.data.cpu().numpy(), stat_3d['mean'], stat_3d['std'], stat_3d['dim_use'])
        outputs_unnorm = data_process.unNormalizeData(outputs.data.cpu().numpy(), stat_3d['mean'], stat_3d['std'], stat_3d['dim_use'])

        # remove dim ignored
        dim_use = np.hstack((np.arange(3), stat_3d['dim_use']))

        outputs_use = outputs_unnorm[:, dim_use]
        targets_use = targets_unnorm[:, dim_use]
        pred_result.append(outputs_unnorm)

        if procrustes:
            for ba in range(inps.size(0)):
                gt = targets_use[ba].reshape(-1, 3)
                out = outputs_use[ba].reshape(-1, 3)
                _, Z, T, b, c = get_transformation(gt, out, True)
                out = (b * out.dot(T)) + c
                outputs_use[ba, :] = out.reshape(1, 51)

        sqerr = (outputs_use - targets_use) ** 2

        distance = np.zeros((sqerr.shape[0], 17))
        dist_idx = 0
        for k in np.arange(0, 17 * 3, 3):
            distance[:, dist_idx] = np.sqrt(np.sum(sqerr[:, k:k + 3], axis=1))
            dist_idx += 1
        all_dist.append(distance)
        
        # update summary
        if (i + 1) % 100 == 0:
            batch_time = time.time() - start
            start = time.time()

        bar.suffix = '({batch}/{size}) | batch: {batchtime:.4}ms | Total: {ttl} | ETA: {eta:} | loss: {loss:.6f}' \
            .format(batch=i + 1,
                    size=len(test_loader),
                    batchtime=batch_time * 10.0,
                    ttl=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg)
        bar.next()

    all_dist = np.vstack(all_dist)
    joint_err = np.mean(all_dist, axis=0)
    ttl_err = np.mean(all_dist)
    bar.finish()
    print (">>> error: {} <<<".format(ttl_err))
    return losses.avg, ttl_err,pred_result

>>> creating model
>>> total params: 4.29M
>>> loading ckpt from '/media/ivan/Ivan/Project Lab/THESIS_FALL_DETECTION/2D to 3D Pose/3d_pose_baseline_pytorch/checkpoint/test/ckpt_best.pth.tar'
>>> ckpt loaded (epoch: 157 | err: 41.94968150886006)


In [47]:
losses, ttl_err,pred_result=test(test_loader, model, criterion, stat_)

>>> error: 41.94968150886006 <<<


In [48]:
pred_result[0][0].shape

(96,)

In [49]:
len(pred_result)

538

In [50]:
test_loader.dataset.test_out

[array([ 1.04370808, -0.49845911,  0.59328969,  0.88429347,  0.68480205,
         0.15843593,  1.03859836,  0.91937987, -0.19850847, -1.04371688,
         0.49846761, -0.59329453, -0.86051911,  0.79886426, -0.28061295,
        -0.78456268,  0.9820355 , -0.70143259,  0.17777876, -0.72562044,
        -0.16302693,  0.14347379, -0.55787823, -0.17855744,  0.02308231,
        -0.92936894,  0.07955362,  0.21924759, -0.86146345, -0.22355783,
        -0.97329683, -0.76380423, -0.88912873, -1.66178986, -1.44375302,
        -1.33910822, -2.6594263 , -1.11529586, -1.27545948,  1.3344863 ,
        -0.95542012,  0.2176612 ,  1.90797545, -1.62298088,  0.54747677,
         2.65410288, -1.10682446,  1.03503916]),
 array([ 1.04384666, -0.49772541,  0.59280658,  0.88452032,  0.68492656,
         0.15798382,  1.03864478,  0.91948974, -0.1989096 , -1.04385546,
         0.4977339 , -0.59281142, -0.86039254,  0.79880839, -0.28058076,
        -0.78433504,  0.98197103, -0.70176448,  0.1799394 , -0.72478531,
  

In [52]:
# # Visualize random samples
# import matplotlib.gridspec as gridspec

# # 1080p	= 1,920 x 1,080
# fig = plt.figure( figsize=(19.2, 10.8) )

# gs1 = gridspec.GridSpec(5, 9) # 5 rows, 9 columns
# gs1.update(wspace=-0.00, hspace=0.05) # set the spacing between axes.
# plt.axis('off')

# subplot_idx, exidx = 1, 1
# nsamples = 15

# Plot 3d Test
ax3 = plt.subplot(gs1[subplot_idx+1], projection='3d')
p3d = pred_result[0][1000]
show3Dpose( p3d, ax3, lcolor="#9b59b6", rcolor="#2ecc71" )

# Load Human Eva Dataset

In [42]:
import numpy as np

humaneva = np.load('../Dataset/HumanEva-I/converted_15j/data_2d_humaneva15_gt.npz',allow_pickle=True)


In [43]:
humaneva.files

['positions_2d', 'metadata']

In [60]:
skeleton_2d=humaneva['positions_2d']

In [61]:
skeleton_2d.item()['Train/S1']

{'Box 1 chunk1': [array([[[316.13177 , 215.19003 ],
          [314.27393 , 142.98988 ],
          [342.1913  , 144.30025 ],
          [346.34967 , 194.21114 ],
          [313.92993 , 187.71114 ],
          [289.73615 , 145.92108 ],
          [284.27673 , 191.82022 ],
          [299.7698  , 168.99707 ],
          [321.39874 , 219.3925  ],
          [328.91653 , 283.83554 ],
          [324.441   , 338.7165  ],
          [305.7589  , 218.26999 ],
          [289.5204  , 283.2632  ],
          [284.12704 , 343.16876 ],
          [312.2871  ,  89.304886]],
  
         [[315.96527 , 215.31413 ],
          [313.88788 , 142.95862 ],
          [342.0176  , 144.27672 ],
          [345.75845 , 194.21503 ],
          [313.4757  , 185.73613 ],
          [289.43652 , 145.98813 ],
          [283.62524 , 192.05972 ],
          [300.4501  , 172.48453 ],
          [321.26825 , 219.42247 ],
          [328.79288 , 283.73114 ],
          [324.40585 , 338.71158 ],
          [305.60745 , 218.40805 ],
        

In [59]:
#2D Items

skeleton_2d.item()['Train/S1']['Box 1 chunk1'][0].shape

(28, 15, 2)

ItemsView(<numpy.lib.npyio.NpzFile object at 0x7f8ad02f5b50>)